## Training Task Adapters
Using randomized search, we identify optimal hyperparameters to train task specfic adapters on GLUE tasks.

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# cd drive/MyDrive/cs7643-deep-learning-summer-2021/

In [3]:
# !pip install -Uqq adapter-transformers datasets

%load_ext autoreload
%autoreload 2

import torch
from time import time
from typing import Dict, List
from task_utils import TaskModelArguments, TaskDataTrainingArguments
from task import train_task_adapter
from transformers import (
    MultiLingAdapterArguments,
    TrainingArguments,
)

### CoLA Task

In [4]:
import random
import itertools
import subprocess
import pandas as pd

def getParams(dictionary, limit):
    paramsList = [dict(zip(dictionary, v)) for v in itertools.product(*dictionary.values())]
    random.shuffle(paramsList)

    if limit is not False:
        paramsList = paramsList[0:min(limit, len(paramsList))]

    return paramsList

def initParse(dictionary: Dict):
    model = TaskModelArguments(
        model_name_or_path=dictionary.get('model_name_or_path')
    )

    data = TaskDataTrainingArguments(
        task_name=dictionary.get('task_name'),
        max_seq_length=dictionary.get('max_seq_length'),
        pad_to_max_length=dictionary.get('pad_to_max_length')
    )

    training = TrainingArguments(
        adam_beta1=dictionary.get('adam_beta1'),
        adam_beta2=dictionary.get('adam_beta2'),
        adam_epsilon=dictionary.get('adam_epsilon'),
        learning_rate=dictionary.get('learning_rate'),
        fp16=dictionary.get('fp16'),
        warmup_ratio=dictionary.get('warmup_ratio'),
        warmup_steps=dictionary.get('warmup_steps'),
        weight_decay=dictionary.get('weight_decay'),
        do_train=dictionary.get('do_train'),
        do_eval=dictionary.get('do_train'),
        per_device_train_batch_size=dictionary.get('per_device_train_batch_size'),
        num_train_epochs=dictionary.get('num_train_epochs'), # CHANGE ME
        overwrite_output_dir=dictionary.get('overwrite_output_dir'),
        output_dir=f"./adapter/task/{dictionary.get('task_name')}",
    )

    adapter = MultiLingAdapterArguments(
        train_adapter=True,
        adapter_config="pfeiffer",
    )

    return model, data, training, adapter

def train(params: Dict) -> List:
    model, data, training, adapter = initParse(params)
    
    train_stats, eval_stats = train_task_adapter(
        model_args=model, 
        adapter_args=adapter, 
        training_args=training, 
        data_args=data
    )
    
    row = []
    row.extend(list(params.values()))
    row.extend(list(train_stats.values()))
    row.extend(list(eval_stats.values()))
    
    header = []
    header.extend(list(params.keys()))
    header.extend(list(train_stats.keys()))
    header.extend(list(eval_stats.keys()))

    output_df = pd.DataFrame([row], columns=header)
    
    del model
    del data
    del training
    del adapter
    
    return output_df

**Define Dictionary of Hyperparameters**

In [ ]:
glue_tasks = [
    #"cola",
    #"mnli",
    #"mrpc",
    #"qnli",
    #"qqp",
    "rte",
    "sst2",
    "stsb",
    "wnli",
]

In [14]:
task = 'cola'
paramDictionary = {
    'task_name':[task],
    'model_name_or_path':['roberta-base'],
    'max_seq_length':[64, 128, 256],
    'pad_to_max_length':[True],
    'per_device_train_batch_size':[16, 32, 64],
    'adam_beta1':[.9],
    'adam_beta2':[.999],
    'adam_epsilon':[1e-8,1e-7,1e-6],
    'fp16':[True],
    'learning_rate':[1e-5,5e-5,1e-4,5e-4,1e-3],
    'warmup_ratio':[0.0],
    'warmup_steps':[0],
    'weight_decay':[0.0],
    'do_train':[True],
    'do_eval':[True],
    'num_train_epochs':[10],
    'overwrite_output_dir':[True],
    'adapter_config':['pfeiffer']
}

**Begin Looping**

In [ ]:
limit = 15 #Numerical or False for no limit

for data_set in glue_tasks:
    paramDictionary["task_name"] = [data_set]
    paramsList = getParams(paramDictionary, limit)

    results = None
    for p in paramsList:
        trial_data = train(p)
        
        if results is not None:
            results = results.append(trial_data)
        else:
            results = trial_data #first pass through the loop
        
        torch.cuda.empty_cache()
    results.to_csv(f"./adapter/task/{data_set}_hp_search.{time():.0f}.csv")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
07/27/2021 09:39:38 - WARNING - task -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
07/27/2021 09:39:38 - INFO - task -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-07,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gre

07/27/2021 09:39:44 - INFO - task -   Sample 83810 of the training set: {'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'idx': 83810, 'input_ids': [0, 2264, 8893, 9, 5, 221, 9788, 58, 382, 8, 248, 9335, 3517, 45, 2460, 7, 3679, 116, 2, 2, 4148, 974, 759, 23, 5, 2238, 4580, 760, 6, 10, 121, 4, 104, 4, 262, 212, 35614, 2925, 6304, 40955, 30456, 2711, 36, 246, 6, 151, 3878, 43, 8, 5, 121, 4, 104, 4, 112, 620, 6144, 2925, 36, 1092, 6, 151, 2383, 2], 'label': 0, 'question': 'What tactics of the PVA were US and ROK troops not prepared to handle?', 'sentence': "On 27 November at the Korean eastern front, a U.S. 7th Infantry Division Regimental Combat Team (3,000 soldiers) and the U.S. 1st Marine Division (12,000–15,000 marines) were unprepared for the PVA 9th Army Group's three-pronged encirclement tactics at the Battle of Chosin 

Using amp fp16 backend
/home/jason/.local/share/virtualenvs/cs7643-deep-learning-summer-2021-YD_jfhWv/lib/python3.8/site-packages/transformers/trainer.py:1052: FutureWarning: `model_path` is deprecated and will be removed in a future version. Use `resume_from_checkpoint` instead.
  warnings.warn(
The following columns in the training set  don't have a corresponding argument in `RobertaModelWithHeads.forward` and have been ignored: sentence, idx, question.
***** Running training *****
  Num examples = 104743
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 32740


Step,Training Loss
500,0.487700
1000,0.375900
1500,0.348200
2000,0.328100
2500,0.325000
3000,0.308700
3500,0.298300
4000,0.284200
4500,0.272800
5000,0.277800


/home/jason/.local/share/virtualenvs/cs7643-deep-learning-summer-2021-YD_jfhWv/lib/python3.8/site-packages/transformers/trainer.py:1351: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
Saving model checkpoint to ./adapter/task/qnli/checkpoint-500
Configuration saved in ./adapter/task/qnli/checkpoint-500/qnli/adapter_config.json
Module weights saved in ./adapter/task/qnli/checkpoint-500/qnli/pytorch_adapter.bin
Configuration saved in ./adapter/task/qnli/checkpoint-500/qnli/head_config.json
Module weights saved in ./adapter/task/qnli/checkpoint-500/qnli/pytorch_model_head.bin
Configuration saved in ./adapter/task/qnli/checkpoint-500/qnli/head_config.json
Module weights saved in ./adapter/task/qnli/checkp

...

07/27/2021 11:57:48 - INFO - /home/jason/.local/share/virtualenvs/cs7643-deep-learning-summer-2021-YD_jfhWv/lib/python3.8/site-packages/datasets/metric.py -   Removing /home/jason/.cache/huggingface/metrics/glue/qnli/default_experiment-1-0.arrow
07/27/2021 11:57:48 - INFO - task -   ***** Eval results qnli *****
07/27/2021 11:57:48 - INFO - task -     eval_loss = 0.2628665268421173
07/27/2021 11:57:48 - INFO - task -     eval_accuracy = 0.9234852645066813
07/27/2021 11:57:48 - INFO - task -     eval_runtime = 16.9653
07/27/2021 11:57:48 - INFO - task -     eval_samples_per_second = 322.01
07/27/2021 11:57:48 - INFO - task -     eval_steps_per_second = 40.259
07/27/2021 11:57:48 - INFO - task -     epoch = 10.0
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info

07/27/2021 11:57:51 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at /home/jason/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c031cbf8f5df7c97.arrow
07/27/2021 11:57:51 - INFO - task -   Sample 83810 of the training set: {'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'idx': 83810, 'input_ids': [0, 2264, 8893, 9, 5, 221, 9788, 58, 382, 8, 248, 9335, 3517, 45, 2460, 7, 3679, 116, 2, 2, 4148, 974, 759, 23, 5, 2238, 4580, 760, 6, 10, 121, 4, 104, 4, 262, 212, 35614, 2925, 6304, 40955, 30456, 2711, 36, 246, 6, 151, 3878, 43, 8, 5, 121, 4, 104, 4, 112, 620, 6144, 2925, 36, 1092, 6, 151, 2383, 2], 'label': 0, 'question': 'What tactics of the PVA were US and ROK troops not prepared to handle?', 'sentence': "On 27 November at the Korean 

Using amp fp16 backend
/home/jason/.local/share/virtualenvs/cs7643-deep-learning-summer-2021-YD_jfhWv/lib/python3.8/site-packages/transformers/trainer.py:1052: FutureWarning: `model_path` is deprecated and will be removed in a future version. Use `resume_from_checkpoint` instead.
  warnings.warn(
The following columns in the training set  don't have a corresponding argument in `RobertaModelWithHeads.forward` and have been ignored: sentence, idx, question.
***** Running training *****
  Num examples = 104743
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 65470


Step,Training Loss
500,0.533500
1000,0.438500
1500,0.418300
2000,0.407600
2500,0.384400
3000,0.392500
3500,0.371600
4000,0.354700
4500,0.387200
5000,0.361600


Saving model checkpoint to ./adapter/task/qnli/checkpoint-500
Configuration saved in ./adapter/task/qnli/checkpoint-500/qnli/adapter_config.json
Module weights saved in ./adapter/task/qnli/checkpoint-500/qnli/pytorch_adapter.bin
Configuration saved in ./adapter/task/qnli/checkpoint-500/qnli/head_config.json
Module weights saved in ./adapter/task/qnli/checkpoint-500/qnli/pytorch_model_head.bin
Configuration saved in ./adapter/task/qnli/checkpoint-500/qnli/head_config.json
Module weights saved in ./adapter/task/qnli/checkpoint-500/qnli/pytorch_model_head.bin
tokenizer config file saved in ./adapter/task/qnli/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./adapter/task/qnli/checkpoint-500/special_tokens_map.json
/home/jason/.local/share/virtualenvs/cs7643-deep-learning-summer-2021-YD_jfhWv/lib/python3.8/site-packages/transformers/trainer.py:1351: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behav

In [ ]:
results

In [12]:
results.to_csv(f"./adapter/task/{data_set}_hp_search.{time():.0f}.csv")